# HCR Plotting Notebook 



# Index of Data Structures in Notebook
   
## DataFrames

#### intDF
    Derived from the full cp output DataFrame (intensityDF)
    Contains the relevant and essential measurements for entire data set 

#### manualDFs (within manual_dict)
    manual counts by image, probe, and counter 
    
#### IIIDF
    dataframe determining the integrated intensity cutoff on an image x counter x probe basis 

#### filteredDF
    contains the number of positive cells for each image based on the median integrated intensity threshold value for a given thresholding method (NumD6 or NumD8)
 
<br> 
    
## Outputted Thresholded DataFrames


_**First group of DFs include just probe 1 positive, probe 2 positive, and double positive. Double positives are counted in probe 1 and 2 positive (and are therefore counted twice).**_

#### Chosen_IIIDF
    Filters data after threshold is chosen to only include information relevant to NumD6 or NumD8. Shows chosen threshold cells that are positive for one probe or double positive 
	
	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive, AP positions, binned AP positions (6 and 9)

#### Animal_IIIDF
    Same data included in Chosen IIIDF, but grouped by animal ID with summed total GFP, probe positive, and proportion positive for each of: probe1, probe2, double positive

	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive

#### APbin_DF
    Same data included in Chosen IIIDF, but groups overlapping AP bins(9) for each animal / injection / probe. Shows total GFP, probe positive, and proportion positive per bin.

	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive, AP bin(9 only)
    
    
<br>

_**DF2s have the same data as previous DF, but double positive cells are removed from the probe 1 positive and probe 2 positive counts. Double negative cells are also added to the counts.**_

#### Chosen_IIIDF2
    Filters data from Chosen IIIDF to remove double positive cells from probe 1 and probe 2 positive counts. Adds in double negative count. 
	
	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive, AP positions, binned AP positions (6 and 9)

#### Animal_IIIDF2
    Same data included in Chosen IIIDF, but grouped by animal ID with summed total GFP, probe positive, and proportion positive for each of: probe1, probe2, double positive as well as proportion negative (still 	under proportion positive column).

	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive

#### APbin_DF2
    Same data included in Chosen IIIDF, but groups overlapping AP bins(9) for each animal / injection / probe. Shows total GFP, probe positive, and proportion positive/negative per bin.

	Includes: FileName_RAW, Animal ID, Injection site, Probe, chosen threshold, total GFP, probe positive cells, proportion positive, AP bin(9 only)

<br>

## Dictionaries

#### projdict
    Dictionary of Mouse IDs (Keys) and associated Projection Types (Values)    
    
#### manual_dict
    dictionary of dataframes of manual counts. Keys indicate which probe and which "stage", i.e. testing or training, 
    the counts are used for 

#### man_images_dict
    dictionary of lists of the specific images in each manual_dict dataframe. Keys are the same as above, just with "_images" added on 
    
#### probes_mean_dict
    Dictionary of probe x thresholding methods (Keys) and their associated mean III (Values)

#### probes_median_dict
    Dictionary of probe x thresholding methods (Keys) and their associated median III (Values)

____________________________

# Importing Packages and Data 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import os
from zipfile import ZipFile
import pandas as pd
import tabulate as tabulate
from pandas.plotting import table

plt.rcParams['svg.fonttype'] = 'none'

## MANUALLY Plug in Probes

In [ ]:
Probe1 = "Slc30a3"
Probe2 = "Spp1"

#If merging multiple datasets with different thresholds, set other thresholds below:
p1_meth = 'Slc30a3_NumD6_norm'
p2_meth = 'Spp1_NumD8_norm'

In [ ]:
if Probe1 == "Kcng2":
    P1NumD="_NumD8"
    P2NumD="_NumD8" ##RK Changed on 10-23-23
elif Probe1 == "Slc30a3":
    P1NumD="_NumD8"
    P2NumD="_NumD6"
elif Probe1 == "Nos1":
    P1NumD="_NumD8"
    P2NumD="_NumD8"
    
if normalize == False:
    probe1_method = Probe1 +"_II"+P1NumD
    probe2_method = Probe2 +"_II"+P2NumD
elif normalize == True:
    probe1_method = Probe1+P1NumD+"_norm"
    probe2_method =Probe2+P2NumD+"_norm"

## MANUALLY Paste in filepaths for Anterior Posterior Positions of Images

In [ ]:
##if not merging AP dictionaries

AP_DF = pd.read_excel("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/AP_positions_SlcSpp1_LM.xlsx")
AP_DF
AP_dict = dict(zip(AP_DF["FileName_RAW"].to_list(),AP_DF["AP_Position"].to_list()))


##Use this cell if there's a need to merge AP dictionaries

AP_DF_LM = pd.read_excel("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/AP_positions_SlcSpp1_LM.xlsx")
AP_DF_NR = pd.read_excel("Z:/RKDATA/ConfocalData/SDC/HCR/RV-dG-GFP_40X_TrueBlack/Data/Slc30a3Cy5_Spp1Cy3_RvGFP/AP_positions_Slc30a3Spp1_LMNameEdit.xlsx")


merge_APDF = [AP_DF_LM, AP_DF_NR]
AP_DF = pd.concat(merge_APDF)

AP_DF
AP_dict = dict(zip(AP_DF["FileName_RAW"].to_list(),AP_DF["AP_Position"].to_list()))


In [ ]:
AP_DF

## MANUALLY Paste in CP Filepaths

In [ ]:
if Probe1 == "Kcng2":
    CP_Cell_level_filepath= 'Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Kcng2_Pax6/CellProfilerOutput/Kcng2Pax6_10-17-23FilteredAllRV.csv'
elif Probe1 == "Slc30a3":
    CP_Cell_level_filepath = "Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/CellProfilerOutput/2024_01_16/Slc30a3Spp1_2024_01_16FilteredAllRV.csv"
elif Probe1 == "Nos1":
    CP_Cell_level_filepath = "/Volumes/ryanKast/RKDATA/ConfocalData/SDC/HCR/RV-dG-GFP_40X_TrueBlack/Data/Nos1Cy5_Spp1Cy3_RvGFP/CellProfilerOutput/1-12-23/higher_NumDs/Nos1_Spp1_1-12-23_higher_NumDsFilteredAllGFP.csv"

CP_Image_level_filepath = CP_Cell_level_filepath[0:CP_Cell_level_filepath.find("FilteredAllRV.csv")] +"Image.csv"



## MANUALLY Paste in thresholding CSVs

### Merging two data sets

In [ ]:
chosen_IIIDF_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/chosen_IIIDF.csv')
animal_IIIDF_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/animal_IIIDF.csv')
APbin_DF_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/APbin_DF.csv')
chosen_IIIDF2_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/chosen_IIIDF2.csv')
animal_IIIDF2_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/animal_IIIDF2.csv')
APbin_DF2_LM = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/APbin_DF2.csv')

chosen_IIIDF_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/chosen_IIIDF.csv')
animal_IIIDF_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/animal_IIIDF.csv')
APbin_DF_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/APbin_DF.csv')
chosen_IIIDF2_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/chosen_IIIDF2.csv')
animal_IIIDF2_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/animal_IIIDF2.csv')
APbin_DF2_NR = pd.read_csv('Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/NR/APbin_DF2.csv')

In [ ]:
merge_chosen = [chosen_IIIDF_LM, chosen_IIIDF_NR]
merge_animal = [animal_IIIDF_LM, animal_IIIDF_NR]
merge_APbin = [APbin_DF_LM, APbin_DF_NR]
merge_chosen2 = [chosen_IIIDF2_LM, chosen_IIIDF2_NR]
merge_animal2 = [animal_IIIDF2_LM, animal_IIIDF2_NR]
merge_APbin2 = [APbin_DF2_LM, APbin_DF2_NR]

chosen_IIIDF = pd.concat(merge_chosen)
animal_IIIDF = pd.concat(merge_animal)
APbin_DF = pd.concat(merge_APbin)

chosen_IIIDF2 = pd.concat(merge_chosen2)
animal_IIIDF2 = pd.concat(merge_animal2)
APbin_DF2 = pd.concat(merge_APbin2)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
chosen_IIIDF

In [ ]:
##confirm that merging worked correctly

pd.set_option("display.max_rows", None, "display.max_columns", None)
APbin_DF

### If not Merging, enter CSV threshold path

chosen_IIIDF = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/chosen_IIIDF.csv")
animal_IIIDF = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/animal_IIIDF.csv")
APbin_DF = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/APbin_DF.csv")

chosen_IIIDF2 = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/chosen_IIIDF2.csv")
animal_IIIDF2 = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/animal_IIIDF2.csv")
APbin_DF2 = pd.read_csv("Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/Thresholded_CSVS/APbin_DF2.csv")

## Define Probe Color Dict

In [ ]:
Double = Probe1+Probe2
Neg = "Double Negative"

probecolordict = {'Kcng2': 'magenta',
                  'Spp1' : 'cyan',
                  'Pax6' : 'cyan',
                  'Nos1' : 'red',
                  'Slc30a3' : 'magenta',
                  'Slc30a3Spp1': 'darkblue',
                  'Nos1Spp1': 'darkblue',
                  'Kcng2Pax6': 'darkblue',
                  'Double Negative': 'gray'}

injcolordict = {'SC': 'red',
                'PRN': 'yellow',
                'MRN': 'limegreen',
                'Gi': 'darkviolet',
                'dmPAG': 'orange',
                'POm': 'skyblue'}

#for barplots in combo bar-swarm plots
probecolordict2 = {'Kcng2': 'magenta',
                  'Spp1' : 'cyan',
                  'Pax6' : 'cyan',
                  'Nos1' : 'red',
                  'Slc30a3' : 'magenta',
                  'Slc30a3Spp1': 'steelblue',
                  'Nos1Spp1': 'steelblue',
                  'Kcng2Pax6': 'steelblue',
                  'Double Negative': 'lightgray'}

## Choose if normalizing analysis by cell area

In [ ]:
#have decided to use normalized by cell area for analysis
normalize = True

if normalize == True:
    norm_save = "Area_normalized"
elif normalize == False:
    norm_save= "not_normalized"
    
image_save_path = "Z:/RKDATA/ConfocalData/FV3000/HCR/HCR_FINAL/Analysis/Slc30a3_Spp1/Python_Analysis/figures/NR_LM_MergedFigures"



## Make a Mouse ID to Projection Target Dictionary

In [ ]:
projdict = {'4-5-4f': 'SC' ,
            '4-5-5': 'SC', 
            '7309-5': 'dmPAG',
            'B1-4a-1R': 'dmPAG',
            '7376-1': 'PRN', 
            '7376-2': 'PRN',
            '1907-2': 'PRN',
            '1907-3': 'PRN',
            'WT1-5b-1R': 'PRN',
            'WT1-5b-none': 'PRN',
            '1907-4': 'SC',
            '1974-8': 'SC',
            'WT1-5a-none': 'SC',
            'WT2-9-1R': 'SC',
            
            ##New cases imaged on FV3000 below:
            'JAX-10F-1R': 'dmPAG',
            'JAX-10F-NE': 'dmPAG',
            'JAX-14F-1R': 'dmPAG',
            'JAX-MM-NE': 'dmPAG',
            'JAX-12F-1L': 'Gi',
            'JAX-12F-1R': 'Gi',
            'JAX-14M-1R': 'Gi',
            'JAX-15M-1L': 'Gi',
            'JAX-12M-1L': 'MRN',
            'JAX-12M-1R': 'MRN',
            'JAX-12M-NE': 'MRN',
            'JAX-14M-1L': 'MRN',
            'JAX-10M-1R': 'POm',
            'JAX-10M-NE': 'POm',
            'JAX-13F-1R': 'POm',
            'JAX-13M-1R': 'POm'}

##13M-1R --> 15M-1R

## Figures by number single pos, double pos, double neg

### Most cells for figures follow the same format as the one below

In [ ]:
# Binning AP coordinates into 9 position bins and six position 6 bins to allow for corser analysis by position

AP_order = chosen_IIIDF["AP_position"].unique().tolist()
AP_order.sort(reverse= True)

In [ ]:
plt.figure(dpi = 300)

#manually change Title that apears above Figure, and image_name that saves a filename 
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Image)"
image_name = "percent-pos_swarm-bar_image-level_hue-inj"
#_____________________________________________-_______----____________________________________________________

#code for figure settings 

MRNpatch = mpatches.Patch(facecolor=injcolordict['MRN'], label = 'MRN', edgecolor = 'black')
Gipatch = mpatches.Patch(facecolor=injcolordict['Gi'], label = 'Gi', edgecolor = 'black')
dmPAGpatch = mpatches.Patch(facecolor=injcolordict['dmPAG'], label = 'dmPAG', edgecolor = 'black')
POmpatch = mpatches.Patch(facecolor=injcolordict['POm'], label = 'POm', edgecolor = 'black')
SCpatch = mpatches.Patch(facecolor=injcolordict['SC'], label = 'SC', edgecolor = 'black')
PRNpatch = mpatches.Patch(facecolor=injcolordict['PRN'], label = 'PRN', edgecolor = 'black')

g = sns.swarmplot(data = chosen_IIIDF[(chosen_IIIDF["Method"] == probe1_method) | 
                                     (chosen_IIIDF["Method"] == probe2_method) |
                                      (chosen_IIIDF["Method"] == p2_meth) |
                                      (chosen_IIIDF["Method"] == p1_meth)] ,
            x = "Probe", y = "Proportion Positive", hue = "Inj Site", dodge=True, palette = injcolordict,
                edgecolor="0",
                linewidth=1, 
                size = 3)

g = sns.barplot(data = chosen_IIIDF[(chosen_IIIDF["Method"] == probe1_method) | 
                                     (chosen_IIIDF["Method"] == probe2_method)|
                                      (chosen_IIIDF["Method"] == p2_meth) |
                                      (chosen_IIIDF["Method"] == p1_meth)],
            x = "Probe", y = "Proportion Positive", hue = "Inj Site", palette = injcolordict,
                saturation = 0.75,
                linewidth=1.5,
                errcolor="0", 
                edgecolor="0",
                errwidth=1.5,
                capsize=.07)

#labeling of figure
plt.ylabel('Proportion Positive')
plt.xlabel('Marker')
plt.ylim(0,1.1)
plt.legend(title= 'ZI Projection Type', loc='upper left', borderaxespad=0, handles = [dmPAGpatch, POmpatch, SCpatch])#, Gipatch, MRNpatch, SCpatch, PRNpatch])
sns.despine(right=True, top =True)
plt.title(Title, pad=20)#"\nNormalized by area = " + str(normalize))

#uncomment the line below to save figure to previously determined save path

#plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
chosen_IIIDF

In [ ]:
plt.figure(dpi = 300)
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Animal)"
image_name = "percent-pos_swarm-bar_animal-level_hue-inj"

dmPAGpatch = mpatches.Patch(facecolor=injcolordict['dmPAG'], label = 'dmPAG', edgecolor = 'black')
POmpatch = mpatches.Patch(facecolor=injcolordict['POm'], label = 'POm', edgecolor = 'black')
Gipatch = mpatches.Patch(facecolor=injcolordict['Gi'], label = 'Gi', edgecolor = 'black')
MRNpatch = mpatches.Patch(facecolor=injcolordict['MRN'], label = 'MRN', edgecolor = 'black')
SCpatch = mpatches.Patch(facecolor=injcolordict['SC'], label = 'SC', edgecolor = 'black')
PRNpatch = mpatches.Patch(facecolor=injcolordict['PRN'], label = 'PRN', edgecolor = 'black')

g = sns.swarmplot(data = animal_IIIDF[(animal_IIIDF["Method"] == probe1_method) | 
                                     (animal_IIIDF["Method"] == probe2_method) |
                                      (animal_IIIDF["Method"] == p2_meth) |
                                      (animal_IIIDF["Method"] == p1_meth)
                                     ] ,
            x = "Probe", y = "Proportion Positive", hue = "Inj Site", dodge=True, palette = injcolordict,
                  linewidth=1.5, 
                  size = 6,
                  edgecolor="0")

g = sns.barplot(data = animal_IIIDF[(animal_IIIDF["Method"] == probe1_method) | 
                                     (animal_IIIDF["Method"] == probe2_method) |
                                      (animal_IIIDF["Method"] == p2_meth) |
                                      (animal_IIIDF["Method"] == p1_meth)],
            x = "Probe", y = "Proportion Positive", hue = "Inj Site",saturation= 0.8, palette = injcolordict,
                 linewidth=1.5, 
                 errcolor="0",
                 errwidth=1.5,
                 edgecolor="0", 
                 capsize=.08)
plt.yticks()
plt.ylim(0,1.1)
plt.ylabel('Proportion Positive')
plt.xlabel('Marker')
plt.legend(title= 'ZI Projection Type', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, handles = [dmPAGpatch, POmpatch, SCpatch])#, Gipatch, MRNpatch, SCpatch, PRNpatch])
plt.title(Title, pad=20)#+"\nNormalized by area = " + str(normalize))
sns.despine(right=True, top =True)
#plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )

In [ ]:
#DOUBLE POS INCLUDED IN SINGLE POS COUNTS


plt.figure(dpi = 300)
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Image)\n (Double Pos Included in Single Pos)"
image_name = "percent-pos_swarm-bar_image-level_hue-probe_3Classes"

probe1patch = mpatches.Patch(facecolor=probecolordict[Probe1], label = Probe1, edgecolor = 'black')
probe2patch = mpatches.Patch(facecolor=probecolordict[Probe2], label = Probe2, edgecolor = 'black')
doublepatch = mpatches.Patch(facecolor='steelblue', label = Double, edgecolor = 'black')

g = sns.swarmplot(data = chosen_IIIDF,
            x = "Inj Site", y = "Proportion Positive", hue = "Probe", dodge=True, 
                  palette = probecolordict,
                    linewidth=1, 
                  size = 2,
                  edgecolor="0")

g = sns.barplot(data = chosen_IIIDF,
                                    x = "Inj Site", 
                                    y = "Proportion Positive", 
                                    hue = "Probe",
                                    saturation= 0.75, 
                                    palette = probecolordict2,
                                    linewidth=1.5,
                                    errwidth=1.5,
                                    errcolor="0",
                                    edgecolor="0", capsize=.07 )

plt.legend(title = 'Marker Gene', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, handles = [probe1patch, probe2patch, doublepatch])
sns.despine(right=True, top =True)
plt.ylim(0,1.1)
plt.title(Title, pad=20)#"\nNormalized by area = " + str(normalize))
plt.xlabel('Projection Type')
plt.ylabel('Proportion Positive')

#plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )

In [ ]:
#double pos and single pos counted sep

plt.figure(dpi = 300)
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Image)"
image_name = "percent-pos_swarm-bar_image-level_hue-probe_3Classes-single_v_double_pos_v_doubleneg"

probe1patch = mpatches.Patch(facecolor=probecolordict[Probe1], label = Probe1, edgecolor = 'black')
probe2patch = mpatches.Patch(facecolor=probecolordict[Probe2], label = Probe2, edgecolor = 'black')
doublepatch = mpatches.Patch(facecolor='steelblue', label = Double, edgecolor = 'black')
negpatch = mpatches.Patch(facecolor='lightgray', label = Neg, edgecolor = 'black')

g = sns.swarmplot(data = chosen_IIIDF2,
            x = "Inj Site", y = "Proportion Positive", hue = "Probe",
                  dodge=True, 
                  palette = probecolordict,
                    linewidth=1, 
                  size = 2,
                  edgecolor="0")

g = sns.barplot(data = chosen_IIIDF2,
                                    x = "Inj Site", 
                                    y = "Proportion Positive", 
                                    hue = "Probe",
                                    
                                    saturation= 0.75, 
                                    palette = probecolordict2,
                                    linewidth=1.5,
                                    errwidth=1.5,
                                    errcolor="0",
                                    edgecolor="0", capsize=.07 )

plt.legend(title = 'Marker Gene', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, 
           handles = [probe1patch, probe2patch, doublepatch,negpatch])
sns.despine(right=True, top =True)
plt.ylim(0,1.1)
plt.title(Title, pad=20)#"\nNormalized by area = " + str(normalize))
plt.xlabel('Projection Type')
plt.ylabel('Proportion Positive')

#plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )

In [ ]:
#DOUBLE POS INCLUDED IN SINGLE POS COUNTS


plt.figure(dpi=300)
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Animal)\n(Double Pos Included in Single Pos)"
image_name = "percent-pos_swarm-bar_animal-level_hue-probe_3Classes"

probe1patch = mpatches.Patch(facecolor=probecolordict[Probe1], label = Probe1, edgecolor = 'black')
probe2patch = mpatches.Patch(facecolor=probecolordict[Probe2], label = Probe2, edgecolor = 'black')
doublepatch = mpatches.Patch(facecolor='steelblue', label = Double, edgecolor = 'black')


g = sns.swarmplot(data = animal_IIIDF ,
                  x = "Inj Site", 
                  y = "Proportion Positive", 
                  hue = "Probe",
                  hue_order = [Probe1,Probe2,Double],
                  dodge=True, 
                  palette = probecolordict,
                  linewidth=1.5, 
                  size = 4,
                  edgecolor="0")

g = sns.barplot(data = animal_IIIDF,
                
                x = "Inj Site", 
                y = "Proportion Positive", 
                hue = "Probe", 
                hue_order = [Probe1,Probe2,Double],
                palette = probecolordict2,
                saturation= 0.75,
                linewidth=1.5, 
                errcolor="0",
                errwidth=1.5,
                edgecolor="0", 
                capsize=.08 )

plt.ylim(0,1.1)
plt.yticks([0,0.2,0.4,0.6,0.8,1.0])
plt.xlabel('Projection Type')
plt.ylabel('Proportion Positive')
plt.legend(title = 'Marker Gene', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, handles = [probe1patch, probe2patch, doublepatch])
plt.title(Title, pad=20)#+"\nNormalized by area = " + str(normalize))
sns.despine(right=True, top =True)
#plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
image_save_path

In [ ]:
#double pos and single pos counted sep

plt.figure(dpi = 300)
Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers (Per Animal)"
image_name = "percent-pos_swarm-bar_animal-level_hue-probe_3Classes-single_v_double_pos"

probe1patch = mpatches.Patch(facecolor=probecolordict[Probe1], label = Probe1, edgecolor = 'black')
probe2patch = mpatches.Patch(facecolor=probecolordict[Probe2], label = Probe2, edgecolor = 'black')
doublepatch = mpatches.Patch(facecolor='steelblue', label = Double, edgecolor = 'black')

g = sns.swarmplot(data = animal_IIIDF2,
            x = "Inj Site", y = "Proportion Positive", hue = "Probe",
                  hue_order = [Probe1,Probe2,Double],
                  dodge=True, 
                  palette = probecolordict,
                    linewidth=1, 
                  size = 5,
                  edgecolor="0")#,
                  #order = ['POm', 'SC', 'dmPAG', 'MRN', 'PRN', 'Gi'])

g = sns.barplot(data = animal_IIIDF2,
                                    x = "Inj Site", 
                                    y = "Proportion Positive", 
                                    hue = "Probe",
                                    hue_order = [Probe1,Probe2,Double],
                                    saturation= 0.75, 
                                    palette = probecolordict2,
                                    linewidth=1.5,
                                    errwidth=1.5,
                                    errcolor="0",
                                    edgecolor="0", capsize=.07)#,
                                    #order = ['POm', 'SC', 'dmPAG', 'MRN', 'PRN', 'Gi'])

plt.legend(title = 'Marker Gene', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, handles = [probe1patch, probe2patch, doublepatch])
sns.despine(right=True, top =True)
plt.ylim(0,1.1)
plt.title(Title, pad=20)#"\nNormalized by area = " + str(normalize))
plt.xlabel('Projection Type')
plt.ylabel('Proportion Positive')

#plt.savefig(image_save_path+"/"+image_name+".png", dpi =300, bbox_inches="tight" )

In [ ]:
sns.set_theme(style = 'ticks', font_scale = 1)

In [ ]:
#double pos and single pos counted sep

plt.figure(figsize = (6,6),dpi=300)
#Title = "Proportion Retrogradely Labeled Neurons\nExpressing ZI Cell Type Markers"
image_name = "percent-pos_swarm-bar_animal-level_hue-probe_3Classes_single_v_double_pos_v_doubleneg"

probe1patch = mpatches.Patch(facecolor=probecolordict[Probe1], label = Probe1, edgecolor = 'black')
probe2patch = mpatches.Patch(facecolor=probecolordict[Probe2], label = Probe2, edgecolor = 'black')
doublepatch = mpatches.Patch(facecolor='steelblue', label = Double, edgecolor = 'black')
negpatch= mpatches.Patch(facecolor='lightgray', label = Neg, edgecolor = 'black')

g = sns.swarmplot(data = animal_IIIDF2 ,
                  x = "Inj Site", 
                  y = "Proportion Positive", 
                  hue = "Probe",
                  hue_order = [Probe1,Probe2,Double, Neg],
                  dodge=True, 
                  palette = probecolordict,
                  linewidth=1.5, 
                  size = 4)#,order = ['POm', 'SC', 'dmPAG', 'MRN', 'PRN', 'Gi'])

g = sns.barplot(data = animal_IIIDF2,
                
                x = "Inj Site", 
                y = "Proportion Positive", 
                hue = "Probe", 
                hue_order = [Probe1,Probe2,Double,Neg],
                palette = probecolordict2,
                linewidth = 1.5,
                edgecolor = 'black',
                saturation= 0.75)#,order = ['POm', 'SC', 'dmPAG', 'MRN', 'PRN', 'Gi'])

plt.ylim(0,0.8)
plt.yticks([0,0.2,0.4,0.6,0.8])
plt.xlabel('Projection Type')
plt.ylabel('Proportion Positive')
plt.legend(title = 'Marker Gene', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, 
           handles = [probe1patch, probe2patch, doublepatch, negpatch])
#plt.title(Title, pad=20)#+"\nNormalized by area = " + str(normalize))
sns.despine(right=True, top =True)
plt.savefig(image_save_path+"/"+image_name+".svg", bbox_inches="tight" )

# Positivity by Anterior-Posterior Position

In [ ]:
AP_table= pd.DataFrame()
AP_table["Rostrocaudal Position Relative to Bregma"] = AP_order

for inj in "dmPAG","POm","Gi","MRN", "SC", "PRN":
    num_images= []
    num_animals= []
    for AP in AP_order:
        temp = chosen_IIIDF[(chosen_IIIDF["Inj Site"] == inj)]
        if AP in temp["AP_position"].to_list():
            num_images.append(len(temp[temp["AP_position"] == AP]["FileName_RAW"].unique()))
            num_animals.append(len(temp[temp["AP_position"] == AP]["Animal ID"].unique()))
        elif AP not in temp["AP_position"].to_list():
            num_images.append(0)
            num_animals.append(0)

    
    AP_table["Number Images: "+inj+" Projecting Cells"] = num_images
    AP_table["Number Animals: " + inj] = num_animals

AP_table

In [ ]:
image_name = "table_images-per-AP-position_by-projection"

styled = AP_table.style.format(precision = 2).hide()
#dfi.export(styled,image_save_path+"/"+image_name+".png")

styled 


In [ ]:
AP9_list = chosen_IIIDF["AP_bin(9)"].unique().tolist()
AP9_list.sort()
APbin9_table= pd.DataFrame()
APbin9_table["Rostrocaudal Bin (9 bins)"] = AP9_list

for inj in "dmPAG","POm","Gi","MRN", "SC", "PRN" :
    num_images= []
    num_animals= []
    for binn in AP9_list:
        temp = chosen_IIIDF[(chosen_IIIDF["Inj Site"] == inj)]
        if binn in temp["AP_bin(9)"].to_list():
            num_images.append(len(temp[temp["AP_bin(9)"] == binn]["FileName_RAW"].unique()))
            num_animals.append(len(temp[temp["AP_bin(9)"] == binn]["Animal ID"].unique()))
        elif binn not in temp["AP_bin(9)"].to_list():
            num_images.append(0)
            num_animals.append(0)

    
    APbin9_table["Number Images: "+inj+" Projecting Cells"] = num_images
    APbin9_table["Number Animals: " + inj] = num_animals

image_name = "table_images-per-AP-9bin_by-projection"

styled = APbin9_table.style.hide()
#dfi.export(styled,image_save_path+"/"+image_name+".png")

styled 

In [ ]:
AP6_list = chosen_IIIDF["AP_bin(6)"].unique().tolist()
AP6_list.sort()
APbin6_table= pd.DataFrame()
APbin6_table["Rostrocaudal Bin (6 bins)"] = AP6_list

for inj in "dmPAG","POm","Gi","MRN", "SC":
    num_images= []
    num_animals= []
    for binn in AP6_list:
        temp = chosen_IIIDF[(chosen_IIIDF["Inj Site"] == inj)]
        if binn in temp["AP_bin(6)"].to_list():
            num_images.append(len(temp[temp["AP_bin(6)"] == binn]["FileName_RAW"].unique()))
            num_animals.append(len(temp[temp["AP_bin(6)"] == binn]["Animal ID"].unique()))
        elif binn not in temp["AP_bin(6)"].to_list():
            num_images.append(0)
            num_animals.append(0)

    
    APbin6_table["Number Images: "+inj+" Projecting Cells"] = num_images
    APbin6_table["Number Animals: " + inj] = num_animals

image_name = "table_images-per-AP-6bin_by-projection"

styled = APbin6_table.style.hide()
#dfi.export(styled,image_save_path+"/"+image_name+".png")

styled 

In [ ]:
#making wide form of chosen_IIIDF2 for graphing purposes

chosenDF_wide = pd.DataFrame()

p1_num_list = []
p2_num_list = []
p1p2_num_list = []
neg_num_list = []

total_GFP_list = []
ID_list = []
image_list = []
inj_list = []
position_list = []
AP_9bin_list = []
AP_6bin_list = []


for image in chosen_IIIDF2["FileName_RAW"].unique():
    x = chosen_IIIDF2[chosen_IIIDF2["FileName_RAW"] == image]
    
    p1_num_list.append(x[x["Probe"] == Probe1]["Cells"].iloc[0])
    p2_num_list.append(x[x["Probe"] == Probe2]["Cells"].iloc[0])
    p1p2_num_list.append(x[x["Probe"] == Double]["Cells"].iloc[0])
    neg_num_list.append(x[x["Probe"] == Neg]["Cells"].iloc[0])
    
    total_GFP_list.append(x["Total GFP"].iloc[0])
    ID_list.append(x["Animal ID"].iloc[0])
    image_list.append(image)
    inj_list.append(x["Inj Site"].iloc[0])
    position_list.append(x["AP_position"].iloc[0])
    AP_9bin_list.append(x["AP_bin(9)"].iloc[0])                   
    AP_6bin_list.append(x["AP_bin(6)"].iloc[0])

chosenDF_wide["FileName_RAW"] = image_list
chosenDF_wide["Animal ID"] = ID_list
chosenDF_wide["Inj Site"] = inj_list
chosenDF_wide["AP_position"] = position_list
chosenDF_wide["AP_bin(9)"] = AP_9bin_list
chosenDF_wide["AP_bin(6)"] = AP_6bin_list
chosenDF_wide["Total GFP"] = total_GFP_list
chosenDF_wide["Num "+Probe1+" Pos"] = p1_num_list
chosenDF_wide["Num "+Probe2+" Pos"] = p2_num_list
chosenDF_wide["Num "+Double+" Pos"] = p1p2_num_list
chosenDF_wide["Num "+Neg] = neg_num_list
chosenDF_wide["Proportion "+Probe1+" Pos"] = chosenDF_wide["Num "+Probe1+" Pos"]/chosenDF_wide["Total GFP"]
chosenDF_wide["Proportion "+Probe2+" Pos"] = chosenDF_wide["Num "+Probe2+" Pos"]/chosenDF_wide["Total GFP"]
chosenDF_wide["Proportion "+Double+" Pos"] = chosenDF_wide["Num "+Double+" Pos"]/chosenDF_wide["Total GFP"]
chosenDF_wide["Proportion "+Neg] = chosenDF_wide["Num "+Neg]/chosenDF_wide["Total GFP"]


chosenDF_wide

In [ ]:
#dataframe with cells from all images of a given projection summated by AP position. Double positives sep from single

dataDF = chosenDF_wide.groupby(["Inj Site","AP_position"]).sum(numeric_only = True).sort_values("AP_position", ascending= False).reset_index()

dataDF["Proportion "+Probe1+" Pos"] = dataDF["Num "+Probe1+" Pos"]/dataDF["Total GFP"]
dataDF["Proportion "+Probe2+" Pos"] = dataDF["Num "+Probe2+" Pos"]/dataDF["Total GFP"]
dataDF["Proportion "+Double+" Pos"] = dataDF["Num "+Double+" Pos"]/dataDF["Total GFP"]
dataDF["Proportion "+Neg] = dataDF["Num "+Neg]/dataDF["Total GFP"]

#dataDF["Check if 1"] = (dataDF["Proportion "+Probe1+" Pos"]+dataDF["Proportion "+Probe2+" Pos"]+
 #                       dataDF["Proportion "+Double+" Pos"]+dataDF["Proportion "+Neg])

dataDF

In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "dmPAG"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##Changed nos1 to slc here 1/20/24
    inj = "POm"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "Gi"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "MRN"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "SC"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "PRN"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "percent-pos-by-AP_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_position", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF[dataDF["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_position",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position (mm)\nRelative to Bregma', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
#dataframe with cells from all images of a given projection summated by AP position. Double positives sep from single

dataDF_9bin = chosenDF_wide.groupby(["Inj Site","AP_bin(9)"]).sum(numeric_only = True).sort_values("AP_bin(9)").reset_index()

dataDF_9bin["Proportion "+Probe1+" Pos"] = dataDF_9bin["Num "+Probe1+" Pos"]/dataDF_9bin["Total GFP"]
dataDF_9bin["Proportion "+Probe2+" Pos"] = dataDF_9bin["Num "+Probe2+" Pos"]/dataDF_9bin["Total GFP"]
dataDF_9bin["Proportion "+Double+" Pos"] = dataDF_9bin["Num "+Double+" Pos"]/dataDF_9bin["Total GFP"]
dataDF_9bin["Proportion "+Neg] = dataDF_9bin["Num "+Neg]/dataDF_9bin["Total GFP"]

#dataDF_9bin["Check if 1"] = (dataDF_9bin["Proportion "+Probe1+" Pos"]+dataDF_9bin["Proportion "+Probe2+" Pos"]+
 #                       dataDF_9bin["Proportion "+Double+" Pos"]+dataDF_9bin["Proportion "+Neg])



In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "dmPAG"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##nos1 changed to slc here 1/20/24
    inj = "POm"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "Gi"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "MRN"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "SC"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "PRN"
    fig = plt.figure(dpi=300)

    ax1 = fig.add_axes([0,0,1,1])
    Title = "Proportion Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position (Binned)"
    image_name = "percent-pos-by-AP9bin_stackedbar_"+inj+"-cells-summed_hue-probe_single_v_double_pos_v_doubleneg"




    choose = ["AP_bin(9)", "Proportion "+Probe1+" Pos", "Proportion "+Probe2+" Pos",
             "Proportion "+Double+" Pos", "Proportion "+Neg]

    dataDF_9bin[dataDF_9bin["Inj Site"] == inj][choose].plot(kind = 'bar', 
                        x= "AP_bin(9)",
                        stacked= True, 
                        ax = ax1,
                        sharex= True,
                        color = [probecolordict[Probe1],probecolordict[Probe2], probecolordict[Double], probecolordict[Neg]]
    )

    plt.legend(bbox_to_anchor=(1.0, 1.0), labels = [Probe1, Probe2, Double, Neg] )

    ax1.set_title(Title, pad = 20)
    ax1.set_xlabel('Approximate Rostrocaudal Position by Bin', labelpad = 10)
    ax1.set_ylabel('Proportion of GFP labeled cells', labelpad = 10)

    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##changed nos1 to slc here 1/20/24
    inj = "dmPAG"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe1, Double, Probe2],
                    palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##changed nos1 to slc here 1/20/24
    inj = "dmPAG"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##changed nos1 to slc here 1/20/24
    inj = "dmPAG"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "POm"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe2, Double, Probe1],
                   palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##changed nos1 to slc here 1/20/24
    inj = "POm"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9,10],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2": ##changed nos1 to slc here 1/20/24
    inj = "POm"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9,10],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "Gi"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe1, Double, Probe2],
                    palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "Gi"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9,10],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "Gi"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9,10],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "MRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "MRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "MRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "SC"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "SC"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Slc30a3" or Probe1 == "Kcng2":
    inj = "SC"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True

if d == True:
    width = 24
if d == False:
    width = 12


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "PRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position"
    image_name = "num-pos-by-AP_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_position', 
                    y='Cells', 
                    hue = 'Probe',
                    order = x_order,
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = "FileName_RAW",
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position (mm)\nRelative to Bregma", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Image"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "PRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )


In [ ]:
d = True
level = "Animal"

if d == True:
    width = 24
elif d == False:
    width = 12
    
if level == "Image":
    UNITS = "FileName_RAW"
elif level == "Animal":
    UNITS = "Animal ID"


if Probe1 == "Nos1" or Probe1 == "Kcng2":
    inj = "PRN"
    fig = plt.figure(figsize = (width,12), dpi=300)

    Title = "Number Retrogradely Labeled "+inj+" Projecting Neurons\nExpressing ZI Cell Type Markers by Rostrocaudal Position\n(Binned/ "+level+" level)"
    image_name = "num-pos-by-APbin9_"+level+"-level_barplot_"+inj+"_hue-probe_single_v_double_pos_v_doubleneg_dodge-"+str(d)


    x_order = chosen_IIIDF2["AP_position"].unique().tolist()
    x_order.sort(reverse= True)

    g = sns.barplot(data = chosen_IIIDF2[chosen_IIIDF2["Inj Site"] == inj],   
                    x='AP_bin(9)', 
                    y='Cells', 
                    hue = 'Probe',
                    order = [1,2,3,4,5,6,7,8,9],
                    hue_order = [Neg, Probe1, Double, Probe2],
                   palette= probecolordict,
                    dodge = d,
                   units = UNITS,
                  )
    plt.title(Title, pad = 20)
    plt.xticks(rotation=90)
    plt.legend(bbox_to_anchor=(1.02, 1))
    plt.xlabel("Approximate Rostrocaudal Position by Bin", labelpad=10)
    plt.ylabel("Cell Count")
    
    #plt.savefig(image_save_path+"/"+image_name+".svg", format= "svg", bbox_inches="tight" )
